In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("의도분류")

LangSmith 추적을 시작합니다.
[프로젝트명]
의도분류


In [10]:
import pandas as pd
from tqdm import tqdm
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda

# 1. 프롬프트 템플릿 정의
intent_classify_prompt = ChatPromptTemplate.from_template("""
당신은 검색 키워드를 분석하는 전문가입니다.  
사용자가 아래와 같은 단어를 웹사이트에서 검색했다면,  
그 사용자가 어떤 목적이나 의도를 가지고 있었는지를 분석해 주세요.

- 키워드는 보통 단어나 짧은 구문으로 되어 있으며, 문맥은 없습니다.
- 가능한 한 간결하고 명확하게, **사용자의 의도를 요약한 카테고리 이름**을 생성해 주세요.
- 이 이름은 데이터 분석이나 분류 작업에서 의도로 쓸 수 있어야 합니다.
- 결과는 5자~15자 이내의 짧고 직관적인 표현으로 작성하세요.
- 불분명하거나 애매할 경우에는 '불명확'으로 작성하세요.
- 비슷한 카테고리 이름은 하나로 통일하시오.

형식:
키워드: [키워드 텍스트]  
의도: [간단한 카테고리 또는 의도 이름]

예시:
키워드: 입양  
의도: 사회복지 이슈

키워드: 김유신  
의도: 인물 정보

키워드: 예총회관  
의도: 장소/시설 조회

키워드: 국무회의록  
의도: 행정 기록 열람

---

다음 키워드에 대해 사용자의 검색 의도를 추론해 주세요:
{user_input}
""")

# 2. LLM 모델 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3. 응답 파싱 함수 정의
def parse_intent_from_text(text: str) -> str:
    try:
        for line in text.split("\n"):
            if line.strip().startswith("의도:"):
                return line.split("의도:")[1].strip()
    except:
        pass
    return "에러"

# 4. 파서와 체인 구성
parser = RunnableLambda(lambda x: type("Obj", (), {"intent": parse_intent_from_text(x.content)}))
intent_classify_chain = intent_classify_prompt | llm | parser

# 5. 엑셀 데이터 불러오기
input_file = "keyword_log.xlsx"
df = pd.read_excel(input_file)

# 6. LLM으로 키워드 의도 추론
predicted_intents = []

for keyword in tqdm(df["KEYWORD"], desc="🔍 의도 분석 중"):
    try:
        output = intent_classify_chain.invoke({"user_input": keyword})
        predicted_intents.append(output.intent)
    except Exception as e:
        print(f"[에러] {keyword}: {e}")
        predicted_intents.append("에러")

# 7. 결과 추가 및 저장
df["predicted_intent"] = predicted_intents

output_file = "intent_classification_full_output.xlsx"
df.to_excel(output_file, index=False)
print(f"\n✅ 결과 저장 완료: {output_file}")


🔍 의도 분석 중: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [14:38<00:00,  1.14it/s]


✅ 결과 저장 완료: intent_classification_full_output.xlsx


In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from collections import Counter

# 🔧 1. 설정
train_csv_path = "train.csv"  # 학습셋 (text, label 포함 CSV)
test_excel_path = "keyword_log_all.xlsx"  # 46,000건 키워드
output_file = "knn_prediction_46000_optimized.xlsx"
embedding_model_name = "intfloat/multilingual-e5-large"

# ✅ 2. 모델 로드
print("🔄 임베딩 모델 로드 중...")
embedding_model = SentenceTransformer(embedding_model_name)

# ✅ 3. 학습 데이터 로딩
print("📥 학습 데이터 로딩 중...")
df_train = pd.read_csv(train_csv_path)
train_sentences = df_train["text"].astype(str).tolist()
train_labels = df_train["label"].tolist()

# ✅ 4. 학습 문장 임베딩
print("📌 학습 문장 임베딩 중...")
train_vectors = embedding_model.encode(train_sentences, convert_to_numpy=True, normalize_embeddings=True)

# ✅ 5. Faiss 인덱스 구성
print("🔍 Faiss 인덱스 생성 중...")
dim = train_vectors.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(train_vectors)

# ✅ 6. 테스트 데이터 로딩
print("📥 테스트 키워드 로딩 중...")
df_test = pd.read_excel(test_excel_path)
test_keywords = df_test["KEYWORD"].astype(str).tolist()

# ✅ 7. 테스트 키워드 임베딩
print("📌 테스트 키워드 임베딩 중...")
test_vectors = embedding_model.encode(test_keywords, convert_to_numpy=True, normalize_embeddings=True)

# ✅ 8. Faiss 검색
print("🔍 KNN 검색 중...")
k = 10
distances, indices = index.search(test_vectors, k)

# ✅ 9. 다수결로 예측 라벨 생성
print("🧠 다수결 예측 중...")
predicted_labels = []
for i in tqdm(range(len(test_keywords))):
    knn_labels = [train_labels[j] for j in indices[i]]
    pred_label = Counter(knn_labels).most_common(1)[0][0]
    predicted_labels.append(pred_label)

# ✅ 10. 결과 저장
df_test["predicted_intent_knn"] = predicted_labels
df_test.to_excel(output_file, index=False)
print(f"\n✅ 완료! 결과 저장됨: {output_file}")
